<a href="https://colab.research.google.com/github/Slebbon/TextGeneration_Projet_PSL_EnC/blob/main/FLAN_Prediction_HIGHRECALL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FLAN MODEL:** Text 2 Text

### **Importation des variables et libraires nécessaires**

In [1]:
! pip install -U accelerate
! pip install -U transformers
! pip install -U torch
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

### **Importation de l'ensemble des donnèes**

In [3]:
# Chargement du tokenizer et du modèle Flan pré-entraîné
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
from datasets import load_dataset
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [5]:
#Chargement des CSV
data_dir = r"C:\Users\marco\OneDrive\Desktop\TXTgeneration\Dataset\Divided_Dataset"

datasets = load_dataset('csv', data_files={
    'train': f'train.csv',
    'validation': f'validation.csv',
    'test': f'test.csv'
})

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
# Fonction de tokenisation des données
def tokenize_function(examples):
    model_inputs = tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)
    model_inputs["labels"] = tokenizer(examples["Author"], padding="max_length", truncation=True, max_length=512)["input_ids"]
    return model_inputs

# Tokeniser les ensembles de données
tokenized_datasets = datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/21054 [00:00<?, ? examples/s]

Map:   0%|          | 0/4513 [00:00<?, ? examples/s]

Map:   0%|          | 0/4512 [00:00<?, ? examples/s]

### **FLAN MODEL:** Entrainement

In [7]:
# Définir les paramètres d'entraînement
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    weight_decay=0.01,
    save_steps=200,                 # sauvegarde un checkpoint tous les 500 pas
    save_total_limit=10,
    num_train_epochs=2,
    predict_with_generate=True
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
# Créer l'entraîneur
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer
)

In [9]:
# Effectuer le fine-tuning
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.004500,0.000253
2,0.001300,0.000097


TrainOutput(global_step=4212, training_loss=1.072587796914102, metrics={'train_runtime': 4544.3751, 'train_samples_per_second': 9.266, 'train_steps_per_second': 0.927, 'total_flos': 7827477372076032.0, 'train_loss': 1.072587796914102, 'epoch': 2.0})

In [10]:
path_to_save_fine_tuned_model = r"C:\Users\marco\OneDrive\Desktop\TXTgeneration"

In [11]:
# Sauvegarder le modèle
model.save_pretrained(path_to_save_fine_tuned_model)
tokenizer.save_pretrained(path_to_save_fine_tuned_model)

('C:\\Users\\marco\\OneDrive\\Desktop\\TXTgeneration/tokenizer_config.json',
 'C:\\Users\\marco\\OneDrive\\Desktop\\TXTgeneration/special_tokens_map.json',
 'C:\\Users\\marco\\OneDrive\\Desktop\\TXTgeneration/spiece.model',
 'C:\\Users\\marco\\OneDrive\\Desktop\\TXTgeneration/added_tokens.json',
 'C:\\Users\\marco\\OneDrive\\Desktop\\TXTgeneration/tokenizer.json')

### **FLAN:** Output

In [12]:
# Chargement du modèle
model = AutoModelForSeq2SeqLM.from_pretrained(path_to_save_fine_tuned_model)
tokenizer = AutoTokenizer.from_pretrained(path_to_save_fine_tuned_model)

In [13]:
# Fonction permettant de générer un texte dans le style souhaité
def generate_text(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [32]:
# Exemple d'utilisation
prompt_mixed = "Hey, how are you?"
generated_text_mixed = generate_text(prompt_mixed)

print("Generated text in mixed style:")
print(generated_text_mixed)

Generated text in mixed style:
Shakespeare


In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
torch.save(model.state_dict(), '/content/drive/MyDrive/model')

### **FLAN:** Evaluation

In [17]:
!pip install -U sacrebleu rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=cd03926843a92c9c36893152126019a653a21a1e1ea153d397ed7d02fc8e5363
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [18]:
from datasets import load_metric

In [19]:
bleu = load_metric("sacrebleu")
rouge = load_metric("rouge")

<ipython-input-19-edf77186883c>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu = load_metric("sacrebleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [20]:
def evaluate_model(model, tokenizer, dataset, max_length=100):
    model.eval()
    preds, labels = [], []

    for example in dataset:
        inputs = tokenizer(example["Text"], return_tensors="pt", padding=True, truncation=True)
        outputs = model.generate(inputs["input_ids"], max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2, early_stopping=True)
        pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
        label = example["Author"]

        preds.append(pred)
        labels.append(label)

    #BLEU
    bleu_score = bleu.compute(predictions=preds, references=[[label] for label in labels])

    #ROUGE
    rouge_score = rouge.compute(predictions=preds, references=labels)

    return bleu_score, rouge_score


In [29]:
test_dataset = tokenized_datasets["test"]
bleu_score, rouge_score = evaluate_model(model, tokenizer, test_dataset)

print(f"BLEU Score: {bleu_score}")
print(f"ROUGE Score: {rouge_score}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:563: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


BLEU Score: {'score': 0.0, 'counts': [4494, 0, 0, 0], 'totals': [4512, 0, 0, 0], 'precisions': [99.60106382978724, 0.0, 0.0, 0.0], 'bp': 1.0, 'sys_len': 4512, 'ref_len': 4512}
ROUGE Score: {'rouge1': AggregateScore(low=Score(precision=0.9937943262411347, recall=0.9937943262411347, fmeasure=0.9937943262411347), mid=Score(precision=0.9960106382978723, recall=0.9960106382978723, fmeasure=0.9960106382978723), high=Score(precision=0.9977836879432624, recall=0.9977836879432624, fmeasure=0.9977836879432624)), 'rouge2': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0)), 'rougeL': AggregateScore(low=Score(precision=0.9940159574468085, recall=0.9940159574468085, fmeasure=0.9940159574468085), mid=Score(precision=0.9960106382978723, recall=0.9960106382978723, fmeasure=0.9960106382978723), high=Score(precision=0.9977836879432624, recall=0.9977836879432624, fmeasure=0.99778368794

**Interprétation Globale**

- **BLEU** : Le score BLEU indique que le modèle génère des textes avec une correspondance modérée au style de référence. La faible précision pour les bi-grammes, tri-grammes et 4-grammes suggère que le modèle ne capture pas bien les structures phrastiques plus longues et complexes, importantes pour le style shakespearien.
- **ROUGE** : Les scores ROUGE1 et ROUGE-L sont élevés, indiquant que le modèle capture bien les mots individuels et leurs séquences immédiates, ce qui est utile pour capturer le style de Trump. Cependant, les scores nuls en ROUGE2 indiquent un manque de cohésion dans les paires de mots consécutifs, un élément important pour la fluidité stylistique de Shakespeare.

In [25]:
!cp -r /content/results /content/drive/MyDrive/

In [26]:
!cp -r /content/Final_model_1 /content/drive/MyDrive/

In [28]:
trainer.save_model("/content/drive/MyDrive/Final_model_2")